In [1]:
pip install pyuppaal --upgrade --user

Requirement already up-to-date: pyuppaal in c:\users\ozgur_ozturk\anaconda3\lib\site-packages (1.0.3)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyuppaal as pyu
def modelVerify(VerifytaPath,ModelPath,NewModelPath,ModelQuery):
    
    pyu.set_verifyta_path(VerifytaPath)
    pipeNet = pyu.UModel(ModelPath)   
    pipeNet.set_queries(ModelQuery)   
    try:
        trace = pipeNet.easy_verify()
        if trace==None:            
            pipeNet = pipeNet.save_as(NewModelPath)   
            pipeNet.set_queries(ModelQuery) 
            trace = pipeNet.easy_verify()
            print(trace)
        else:
            print("Veriyf")
    except Exception as e:        
        print("ErrorVeriyf")


In [ ]:
#guard transaction güncelleme

def systemUpdate(ModelPath,NewWriteModelPath,ModelSystem):

    # XML dosyasını açın ve kök öğeyi alın
    tree = ET.parse(ModelPath)
    root = tree.getroot()
    
    # İlk <transition> etiketini bulun
    system = root.find("system").text
    root.find("system").text=ModelSystem
    tree.write(NewWriteModelPath)


In [ ]:
#transaction silme
def transactionUpdate(ModelPath,TesDirection):
    # XML dosyasını açın ve kök öğeyi alın
    tree = ET.parse(ModelPath)
    root = tree.getroot()

    # İlk <transition> etiketini bulun

    transitionss = root.findall(".//transition") 
    #print(len(transitionss))
    maincount=-1
    transactionCount=len(transitionss)
    while (transactionCount>maincount):
        maincount=maincount+1
        #print(maincount)
        getRoot(maincount,ModelPath,TesDirection)

def getRoot(target,ModelPath,TesDirection):
    # XML dosyasını açın ve kök öğeyi alın
    tree = ET.parse(ModelPath)
    root = tree.getroot()
    templates = root.findall(".//template")
    count=-1
    for template in templates:        
        if str(template.find("name").text) != "Microservice":
            #print(template.find("name").text)
            #Transaction
            transitions = template.findall("transition")    
            for transition in transitions:
                count=count+1
                if count==target:
                    template.remove(transition)
                    break
                
    tree.write(TesDirection+"transactions\\"+str(target)+".xml")

In [4]:
def variable_find(text,start,end):
    # Metni satırlara ayırın
    satirlar = text.splitlines()
    
    # Başlangıç ve bitiş işaretlerini tanımlayın
    baslangic = start
    bitis = end
    
    # İstenen metni toplamak için bir bayrak tanımlayın
    metin_toplandi = False
    alinan_metin = []

    # Her satırı kontrol edin
    for satir in satirlar:
        satir=satir.strip()
        if satir == baslangic:
            metin_toplandi = True
            continue
        elif satir == bitis:
            metin_toplandi = False
            break

        if metin_toplandi:
            alinan_metin.append(satir)

    # İstenen metni birleştirin
    sonuc_metni = "\n".join(alinan_metin)

    # Sonucu görüntüleyin
    return sonuc_metni

def getXmlVariables(MODELPATH):
    
    locations={}
    clocks={
        "0":"t(0)",
        "1":"clk",
    }
    globalVariables={}
    transactions={}
    
    # XML dosyasını parse et
    tree = ET.parse(MODELPATH)
    root = tree.getroot()



    #declaration satırlara ayırma
    declaration = root.find("declaration").text
    variable_count =0;
    global_variables = variable_find(declaration,"//Parameter_Started","//Parameter_Ended")
    global_variables=global_variables.replace("int ","").replace(";","").strip().split(",")
    for global_variable in global_variables:
        globalVariables[str(variable_count)]=global_variable
        variable_count=variable_count+1

    
    #system satırlara ayırma
    system = root.find("system").text
    system_count =0
    clock_count=2
    template_count=0
    sytem_variables = variable_find(system,"//System_Started","//System_Ended")
    sytem_variables=sytem_variables.replace("system ","").replace(";","").strip().split(",")
    for sytem_variable in sytem_variables:
        clocks[str(clock_count)]=sytem_variable
        clock_count=clock_count+1

        temp_location={}
        temp_location["name"]=sytem_variable
        locations[str(template_count)]=temp_location
        template_count=template_count+1


    # Tüm template'ların altındaki transition bilgilerini al
    templates = root.findall(".//template")

    #location
    template_count=-1
    temp_loc={}

    for template in templates:
        for loc in locations:
            if locations[loc]["name"].find(template.find("name").text)!=-1:
                template_count=template_count+1
                indexL=0
                location_value = template.findall("location") 
                temp_L={}
                temp_L["name"]=locations[loc]["name"]

                indexT=0            
                transitions_value = template.findall("transition")
                temp_tra={}

                for t in transitions_value:  
                    temp_T={}
                    source_ref = t.find("source").get("ref")
                    target_ref = t.find("target").get("ref")
                    temp_T["source"]=str(source_ref).replace("id","")
                    temp_T["target"]=str(target_ref).replace("id","")              
                    temp_tra[str(indexT)]=temp_T
                    indexT=indexT+1
                transactions[str(template_count)]=temp_tra

                for l in location_value: 
                    nameL = l.find("name").text 
                    temp_L[str(indexL)]=nameL
                    temp_L[str(l.get('id'))]=nameL
                    indexL = indexL+1; 
                temp_loc[str(template_count)]=temp_L


    locations=temp_loc
    global global_locations
    global global_clocks
    global global_globalVariables
    global global_transactions
    
    global_locations=locations    
    global_clocks=clocks    
    global_globalVariables=globalVariables    
    global_transactions=transactions

    #print("clocks")
    #print(clocks)
    #print("location")
    #print(locations)
    #print("globalVariables")
    #print(globalVariables)
    #print("transactions")
    #print(transactions)

In [16]:

def getXTR(TracePath):
    dosya_adi = TracePath  # Okunacak dosyanın adını burada belirtin

    try:
        count = 0

        with open(dosya_adi, "r") as dosya:
            for satir in dosya:
                #print(satir.strip())  # strip() ile satır sonundaki boşlukları ve satır sonu karakterini kaldırabilirsiniz
                satir=satir.strip()
                
                if satir != ".":
                    values = satir.split(" ")
                    if count==1:
                        if values is not None:
                            #print("Clock;")
                            clockValue=float(values[2])
                            if clockValue !=0 and global_clocks[values[1]]!=0:
                                if clockValue>0:
                                    print(str(global_clocks[values[0]]+"-"+global_clocks[values[1]]+"<="+str((clockValue/2))))
                                elif clockValue<0 and global_clocks[values[1]]!=0:
                                    clockValue=clockValue
                                    print(str(global_clocks[values[0]]+"-"+global_clocks[values[1]]+"<="+str((clockValue/2))))
                                elif clockValue==0 and global_clocks[values[1]]!=0 :
                                    print(str(global_clocks[values[0]]+"-"+global_clocks[values[1]]+"<="+str((clockValue/2))))                              

                            else:
                                print(str(global_clocks[values[0]]+"-"+global_clocks[values[1]]+"<="+str((clockValue/2))))
                                #tempp=count
                           
                        count=0
                    elif count==2:
                        #print("globalVariable")
                        #print(values)
                        print(global_globalVariables["0"]+"=>"+str(values[0]))
                        print(global_globalVariables["1"]+"=>"+str(values[1]))
                        print(global_globalVariables["2"]+"=>"+str(values[2]))
                        print(global_globalVariables["3"]+"=>"+str(values[3]))
                        print(global_globalVariables["4"]+"=>"+str(values[4]))
                        print(global_globalVariables["5"]+"=>"+str(values[5]))
                        print("________________________________________________________________________")
                    elif count==3 and satir.find(";")!=-1:
                        values = satir.split(";")
                        if values is not None:
                            for value in values:
                                value=value.strip()                                
                                if value!=".":
                                    tempValue = value.split(" ")
                                    print("Location: "+str(global_locations[str(tempValue[0])]["name"]))
                                    #print(tempValue)
                                    print("Transaction: Source: "+str(global_locations[str(tempValue[0])]["id"+str(global_transactions[str(tempValue[0])][str(tempValue[1])]["source"])]) +"-> Target: "+ str(global_locations[str(tempValue[0])]["id"+str(global_transactions[str(tempValue[0])][str(tempValue[1])]["target"])]) )
                                    #print("------------------------------------------------------")
                        #print("transaction")
                        count=0
                    else:                    
                        if values is not None:
                            #print("Location;")
                            print("BusinessProcess=>"+str(global_locations["0"][values[0]]))
                            print("Gateway1=>"+str(global_locations["1"][values[1]]))
                            print("Microservice1=>"+str(global_locations["2"][values[2]]))
                            print("Microservice2=>"+str(global_locations["3"][values[3]]))
                            print("Microservice3=>"+str(global_locations["4"][values[4]]))


                        count=0
                else:
                    count=count+1


    except FileNotFoundError:
        print(f"{dosya_adi} dosyası bulunamadı.")
    except Exception as e:
        print("Bir hata oluştu:", e)


In [6]:
def guardInvarimentUpdate(NewWriteModelPath,TesDirection):
    tree = ET.parse(NewWriteModelPath)
    root = tree.getroot()   
    
    clk=3000
    for a in range(10, clk,50):
        for b in range(100, clk,50):
            if b>a:
                ModelSystem=f"""
                BusinessProcess1 = BusinessProcess(0,{b},{a});
                Gateway1=Gateway(0,{500},{10},{550},{600},{50},{5}); //ilk parametre = microservis id ikinci requestin başlama süresi circuit breaker timeout göre belirlendi
                Microservice1=Microservice(0,1,{300},{10});// microservice id/ gideceği microservis id/ request zamanı
                Microservice2=Microservice(1,2,{300},{10});
                Microservice3=Microservice(2,2,{300},{10});
                //System_Started
                system BusinessProcess1,Gateway1,Microservice1,Microservice2,Microservice3;
                //System_Ended
                """
                root.find("system").text=ModelSystem
                tree.write(TesDirection+"guadInvariant\\"+str(a)+"-"+str(b)+".xml")

In [17]:
import xml.etree.ElementTree as ET
import pyuppaal as pyu
import re
import random

ModelPath="tolgaHocam.xml" #Ana model
NewModelPath="new-tolgaHocam.xml" #Verify da oluşturulan Query olan path
TracePath="tolgaHocam-1.xtr" #Trace dosyasını okuyacağımız path
VerifytaPath = ".\\uppaal\\bin-Windows\\verifyta.exe" #Uppaal Verify Path
ModelQuery='A<>(BusinessProcess1.END and clk=<2500)' #Querymiz
#ModelQuery='E<>(BusinessProcess1.END and clk<=2500)' #Querymiz transaction sildiğimizde xtr dosyası çıkanlar başarı olduğumuzu gösterir. Eğer başarısız olursak "A<>(BusinessProcess1.END and clk<1500)" kullanırız ve nerede transaction kesilmiş buluruz.

TesDirection =".\\testcase\\Microservice3\\"
NewWriteModelPath=TesDirection+"Microservice3.xml"

global_locations={}
global_clocks={}
global_globalVariables={}
global_transactions={}
global value_a
global value_b
global value_c
    
ModelSystem=f"""BusinessProcess1 = BusinessProcess(0,{100},{17});
Gateway1=Gateway(0,{500},{10},{550},{600},{50},{5}); //ilk parametre = microservis id ikinci requestin başlama süresi circuit breaker timeout göre belirlendi
Microservice1=MicroserviceFail(0,1,{300},{10});// microservice id/ gideceği microservis id/ request zamanı
Microservice2=Microservice(1,2,{300},{10});
Microservice3=Microservice(2,2,{300},{10});
//System_Started
system BusinessProcess1,Gateway1,Microservice1,Microservice2,Microservice3;
//System_Ended"""


# import the time module
from datetime import datetime
su_an1 = datetime.now()
modelVerify(VerifytaPath,ModelPath,NewModelPath,ModelQuery) #sistemi test etmek için kullanıyorum yeni model ve xtr dosyası çıkarır.
#systemUpdate(ModelPath,NewWriteModelPath,ModelSystem) #sistemi update ediyorum böylece asağıdaki method ile fail olan microservicelerin transactionlarını sileceğim
#transactionUpdate(NewWriteModelPath,TesDirection)#fail olan microservicelerin transactionlarını sileceğim
#guardInvarimentUpdate(NewWriteModelPath,TesDirection)
getXmlVariables(ModelPath)
getXTR(TracePath)

su_an2 = datetime.now()
a =(su_an2-su_an1).total_seconds()*1000
print(a)

#Not her mikroservicede 18 transaction çıktı bunların 6 sı mükkerrerdi. bu nednele 12 kaldı. 52 tanede transaction çıktı.toplam 64 tane çıktı.

ErrorVeriyf
BusinessProcess=>START
Gateway1=>START
Microservice1=>START
Microservice2=>START
Microservice3=>START
t(0)-clk<=0.0
clk-BusinessProcess1<=0.0
BusinessProcess1-Gateway1<=0.0
Gateway1-Microservice1<=0.0
Microservice1-Microservice2<=0.0
Microservice2-Microservice3<=0.0
Microservice3-t(0)<=0.0
response=>0
id=>0
target=>0
delay=>0
temperrorrate=>0
gtwcount=>0
________________________________________________________________________
BusinessProcess=>BeforeGatewayRequest
Gateway1=>START
Microservice1=>START
Microservice2=>START
Microservice3=>START
t(0)-clk<=0.0
clk-t(0)<=100.5
clk-BusinessProcess1<=0.0
BusinessProcess1-Gateway1<=0.0
Gateway1-Microservice1<=0.0
Microservice1-Microservice2<=0.0
Microservice2-Microservice3<=0.0
Microservice3-clk<=0.0
response=>0
id=>0
target=>0
delay=>0
temperrorrate=>0
gtwcount=>0
________________________________________________________________________
Location: BusinessProcess1
Transaction: Source: START-> Target: BeforeGatewayRequest
BusinessProce

In [ ]:
#TÜM XML ÖZETLEME KISMI


def get_transition_info(template):
    indexT=-1
    indexL=-1
    
    #Location
    locations = template.findall("location")
    for location in locations:
        indexL=indexL+1;
        
        nameL = location.find("name").text      
        invariant =location.find("label[@kind='invariant']") 
        if invariant is not None:
            invariant_text = invariant.text.strip()
            print(f"index:{indexL} Name:{nameL}  invariant: {invariant_text}")
        else:
            print(f"index:{indexL} Name:{nameL}")
        

    print("Transaction") 
    #Transaction
    transitions = template.findall("transition")
    for transition in transitions:
        indexT=indexT+1;        
        source_ref = transition.find("source").get("ref")
        target_ref = transition.find("target").get("ref")
        
        if source_ref is not None and target_ref is not None:            
            guard_label = transition.find("label[@kind='guard']")
            synchronisation_label = transition.find("label[@kind='synchronisation']")
            assignment_label = transition.find("label[@kind='assignment']")
            transactionText=f"index:{indexT} Source: {source_ref}, Target: {target_ref}"
                
            if guard_label is not None:
                guard_text = guard_label.text.strip()
                transactionText= transactionText+ f" Guard: {guard_text} "
            if synchronisation_label is not None:
                synchronisation_text = synchronisation_label.text.strip()
                transactionText= transactionText+ f" Synchronisation: {synchronisation_text} "
            if assignment_label is not None:
                assignment_text = assignment_label.text.strip()  
                transactionText= transactionText+ f" Assignment: {assignment_text}"
            
            print(transactionText) 
            

# XML dosyasını parse et
tree = ET.parse("tolgaHocam.xml")
root = tree.getroot()


#declaration
declaration = root.find("declaration").text
print("Global Declaration:" + declaration)


#system
system = root.find("system").text
print("System: " + system)

# Query'leri al
queries = root.findall("queries/query")
print("Query")
for query in queries:
    formula = query.find("formula").text
    print("Formula: " + formula)

# Tüm template'ların altındaki transition bilgilerini al
templates = root.findall(".//template")

for template in templates:
    print("------------------------------------------")
    print("")
    print(template.find("name").text)
    print("Parameter:" + template.find("parameter").text)
    print("Declaration:" + template.find("declaration").text)
    print("")
    get_transition_info(template)

In [ ]:
import xml.etree.ElementTree as ET
import os

def xml_dosyalari_karsilastir(dosya1, dosya2):

    try:
        # İlk XML dosyasını açın ve ağacı oluşturun
        tree1 = ET.parse(dosya1)
        root1 = tree1.getroot()
        #root1.find("system").text="ModelSystem"

        # İkinci XML dosyasını açın ve ağacı oluşturun
        tree2 = ET.parse(dosya2)
        root2 = tree2.getroot()
        #root2.find("system").text="ModelSystem"
        # Ağaçları karşılaştırın
        return ET.tostring(root1) == ET.tostring(root2)

    except Exception as e:
        #print("XML dosyalarını karşılaştırırken bir hata oluştu:", str(e))
        return False



# Klasör yolu
klasor_yolu1 = ".\\testcase\\Microservice1\\transactions"
klasor_yolu2 = ".\\testcase\\Microservice1\\transactions"
# Klasör içindeki dosyaları listele
dosya_listesi1 = os.listdir(klasor_yolu1)
dosya_listesi2 = os.listdir(klasor_yolu2)

# Dosyaları ekrana yazdır
for dosya1 in dosya_listesi1:
    for dosya2 in dosya_listesi2:
        if str(dosya1) !=str(dosya2):         
            sonuc = xml_dosyalari_karsilastir(klasor_yolu1+"\\"+str(dosya1),klasor_yolu2+"\\"+str(dosya2))
            if sonuc:
                print(str(dosya1)+"|"+str(dosya2)+"XML dosyaları aynı.")






In [12]:
import xml.etree.ElementTree as ET
import os


VerifytaPath = ".\\uppaal\\bin-Windows\\verifyta.exe" #Uppaal Verify Path
ModelQuery='A<>(BusinessProcess1.END and clk<=2500)' #Querymiz
#ModelQuery='E<>(BusinessProcess1.END and clk<=2500)' #Querymiz
#ModelQuery='E<> deadlock' #Querymiz

TesDirection =".\\testcase\\Microservice3\\guadInvariantVerify\\clockVerify\\"
# Klasör yolu
klasor_yolu = ".\\testcase\\Microservice3\\guadInvariant\\"

# Klasör içindeki dosyaları listele
dosya_listesi1 = os.listdir(klasor_yolu)


# import the time module
from datetime import datetime
su_an1 = datetime.now()
verifyCount=0
# Dosyaları ekrana yazdır
for dosya in dosya_listesi1:
    try:
      #print(dosya)
      modelVerify(VerifytaPath,klasor_yolu+dosya,TesDirection+dosya,ModelQuery)
    except Exception as e:
        verifyCount=verifyCount+1
        print(verifyCount)
        #Verify olma durumu
        print(e)
    
    
    
su_an2 = datetime.now()
a =(su_an2-su_an1).total_seconds()*1000
print(a)
print(verifyCount)


None
None
None
None
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
None
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
None
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
None
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeriyf
ErrorVeri

In [ ]:
import pyuppaal as pyu   
VerifytaPath = ".\\uppaal\\bin-Windows\\verifyta.exe" #Uppaal Verify Path
ModelQuery='A<>(BusinessProcess1.END and clk<=2500)' #Querymiz
#ModelQuery='E<>(BusinessProcess1.END and clk<=2500)' #Querymiz
#ModelQuery='E<> deadlock' #Querymiz

TesDirection =".\\"
# Klasör yolu
klasor_yolu = ".\\"

pyu.set_verifyta_path(VerifytaPath)
pipeNet = pyu.UModel(klasor_yolu+"10-1150.xml")   
pipeNet = pipeNet.save_as(TesDirection+"10-1150new.xml") 
pipeNet.set_queries(ModelQuery) 
trace = pipeNet.easy_verify()
print(trace)